<a href="https://colab.research.google.com/github/cheng1103/Test/blob/main/Cnyes_category_fast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><span style="font-size:20px"><strong>鉅亨網新聞主題爬蟲（快速版，用於Colab平台）</strong></span></h1>

<p>可自訂主題新聞與日期區間爬取新聞內文，並可設定是否爬取頭條以外的新聞<br />
<span style="color:null"><strong>輸出資料的欄位依序為：</strong></span> <strong>張貼時間、新聞編號(網址)、版別、標題、頭條標記、內文、股號(如果有)</strong> <strong>。資料格式為csv檔，並以UTF8編碼，直接存入Google雲端硬碟</strong></p>

<p>支援的分類新聞目錄：</p>

<p>1.台股 V<br />
&nbsp; a.台灣政經 V<br />
&nbsp; b.台股盤勢 V<br />
&nbsp; c.台股盤勢 V<br />
&nbsp; d.台股盤勢 V</p>

<p>2.國際股 V<br />
&nbsp; a.美股 V<br />
&nbsp; b.美股雷達 V<br />
&nbsp; c.歐亞股 V<br />
&nbsp; d.國際政經 V</p>

<p>3.陸港股 V</p>

<p>4.外匯 V</p>

<p>5.期貨 V</p>

<p>6.房產 V</p>

<p>7.基金 V</p>

<p>8.區塊鏈 V</p>


# 新聞類別與查詢區間設定


In [1]:
#設定區域
#==============================================
#頭條設定 
# 1表示限定頭條； 0表示不限 
Headline = 1

#輸入類別
c = '台股'

#查詢區間
s_time = "2021-01-01 00:00:00" #開始日期 
e_time = "2021-01-31 00:00:00" #結束日期

#檔名設定
file_name = 'TW stock original_data'

#==============================================
data = []


kind = {
  '台股':'tw_stock',
  '台灣政經':'tw_macro',
  '台股盤勢':'tw_quo',
  '專家觀點':'stock_report',
  '台股新聞':'tw_stock_news',

  '國際股':'wd_stock',
  '美股':'us_stock',
  '美股雷達':'us_stock_live',
  '歐亞股':'eu_asia_stock',
  '國際政經':'wd_macro',

  '陸港股':'cn_stock',
  '外匯':'forex',
  '期貨':'future',
  '房產':'cnyeshouse',
  '基金':'fund',
  '區塊鏈':'bc'

}

c = kind.get(c)


#頭條篩選
if Headline == 1 :
  CH = '&isCategoryHeadline=1'
else :
  CH = ''

# 安裝並載入套件

In [2]:
!pip install fake_useragent
from fake_useragent import UserAgent
import requests
from bs4 import BeautifulSoup
from lxml import etree

import random
import time
import datetime
import json
import pandas as pd
import re
#%reset -f

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=b7e1a30a6bcf07a9b14cbc5e44985932ec37fd1ea5fe953007fcc3836377c523
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


# 自定義程式

In [3]:
def url_list(s_time,e_time):
  

  #時間區間設定
  s_time = int(time.mktime(time.strptime(s_time, "%Y-%m-%d %H:%M:%S")))-28800 # 轉成時間戳
  e_time = int(time.mktime(time.strptime(e_time, "%Y-%m-%d %H:%M:%S")))-28800


  p = 1
  last_page = 3
  data = pd.DataFrame()

  while p <= last_page :

    headers = UserAgent().random #隨機請求表頭
    
    url = 'https://api.cnyes.com/media/api/v1/newslist/category/'+c+'?startAt='+str(s_time)+'&endAt='+str(e_time)+CH+"&limit=30&page="+str(p)
    req = requests.get(url, headers={"User-Agent":headers})
    req = req.json()

    #最末頁之路徑
    if p == 1:
      last_page = req["items"]["last_page"] 

    req_data = req["items"]["data"]
    temp = pd.DataFrame.from_dict(req_data)
    
    #擷取資料
    publishAt = temp['publishAt']
    newsId_url = temp['newsId']
    category = temp['categoryName']
    title = temp['title']
    Headline = temp['isCategoryHeadline']
    stock = temp['stock']
    content = temp['content']

    temp_data = pd.concat([publishAt, Headline, category, title, content, stock, newsId_url], axis=1)
    temp_data = reFormat(temp_data)

    data = pd.concat([data, temp_data], ignore_index=True)
    


    #休眠
    t = random.uniform(0.005, 0.1) #隨機數
    time.sleep(t) #暫停

    p = p+1

    if p%20 == 0:
      print(p/last_page*100)
      data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
      t = random.uniform(0.2, 1) #隨機數
      time.sleep(t)


  data.drop_duplicates(subset='newsId', keep='first', inplace=True) #移除重複資料
  return(data)

#==============================================

def reFormat (temp_data):
  for r in range(len(temp_data)):

    #擷取文字
    c = temp_data['content'][r]
    c = etree.HTML(c).xpath('string(.)')
    c = etree.HTML(c).xpath('//*[not(figcaption)]/p/text()') #####
    c = "".join(c)
    c = c.replace('\n', '').replace('\xa0', '')

    temp_data['content'][r] = c


    #擷取股票代號
    temp_data['stock'][r] = ','.join(temp_data['stock'][r])
    if len(temp_data['stock'][r]) == 0 :
      temp_data['stock'][r] = '0'

    #轉換時間格式
    struct_time = time.localtime(temp_data['publishAt'][r]) # 轉成時間元組
    temp_data['publishAt'][r] = time.strftime("%Y/%m/%d %H:%M", struct_time) # 轉成字串

  return(temp_data)

# 執行程式

In [4]:
data = url_list(s_time,e_time)
data = data.sort_values(by='publishAt', ascending=True)
data = data.reset_index(drop=True) 

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

43.47826086956522
86.95652173913044


# 檢視資料

In [5]:
data

,publishAt,isCategoryHeadline,categoryName,title,content,stock,newsId
0,2021/01/01 03:46,1,台股新聞,〈潛力股〉受惠車市快速復甦及電動車產業起飛 敬鵬營運不淡,全球車市需求可望因新冠肺炎疫苗研發有成帶動下，進一步扭轉過去淡季出貨下滑的頹勢，去年 12 ...,2355,4555666
1,2021/01/01 03:48,1,台股新聞,〈潛力股〉和大搭電動車成長順風車 獲利動能加溫,汽車傳動系統大廠和大 (1536-TW) 11 月合併營收 6.02 億元創 25 個月來新...,1536,4555657
2,2021/01/01 03:50,1,台股新聞,〈潛力股〉閎暉調整營運體質見效 明年挑戰擺脫連5年虧損,閎暉 (3311-TW) 近年積極調整體質，將過去手機按鍵業務比例降低，聚焦在車用、穿戴裝置...,3311,4555641
3,2021/01/01 03:51,1,台股新聞,〈潛力股〉環宇-KY基站回溫、轉投資發酵 營運拚反彈,環宇 - KY(4991-TW) 去年受中國基礎建設招標遞延影響，加上轉投資晶成半導體、常州...,4991,4555695
4,2021/01/01 03:56,1,台股新聞,〈潛力股〉奇力新電感、LTCC需求展望佳 併帛漢挹注長期獲利動能,泛國巨 (2327-TW) 集團電感廠奇力新 (2456-TW) 受惠 5G 手機、伺服器、...,"2327,2456,2375",4555707
...,...,...,...,...,...,...,...
1367,2021/01/31 04:42,1,台股新聞,建大產能全線滿載 小車胎需求看到下半年,輪胎廠建大 (2106-TW) 受惠輪胎市場需求持續熱絡，各類車胎產能全滿，自行車胎、機車胎...,2106,4564400
1368,2021/01/31 04:43,1,台股新聞,MIH聯盟開發者工具平台正式上線 3月將召開首次合作夥伴會議,鴻海 (2317-TW) 主導的 MIH 聯盟今 (31) 日正式發表第一代電動車開發者工具...,"2317,2201",4564411
1369,2021/01/31 04:45,1,台股新聞,大立光1月營收、台股鼠年封關 本周大事預告,隨著台美供應鏈合作越趨緊密，經濟部長王美光也將在周五上午率領官員出席台美供應鏈首場對話，台積...,"2317,3008,2330,2454,2881,5820",4564412
1370,2021/01/31 05:10,1,台股新聞,〈誠美材展望〉擬提前全面改選董事 總座將入列董事會,偏光板廠誠美材 (4960-TW) 經歷去年谷底後，營運逐步好轉，不過因應去年法人董事群創 ...,"4960,3481",4564397


# 連結Google Drive並匯出資料

In [6]:
from google.colab import drive
drive.mount('/content/drive')


#輸出檔案

path = '/content/drive/MyDrive/DATA set/'+file_name+'_fast.csv'
data.to_csv(path, index=None)

Mounted at /content/drive
